In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pygpu

%matplotlib inline

## Классификация изображений

Требуется написать программу, отличающую картинки размером 32х32 с часами от картинок с крокодилами. 

В предложенном датасете находилось 1000 картинок (500 с часами и 500 с крокодилами). Для обучения сети были взяты 800 картинок, на оставшихся 200 картинок же проводилось тестирование модели. Так как датасет довольно мал, то используя 800 тренировочных картинок были получены дополнительные примеры, путем поворота их на рандомный угол, а также их отзеркаливание. Нормирование данных также немного улучшило результат. 

Для вдохновения были использованы следующие ссылки:

1. Курс машинного обучения в МФТИ. Пример написания сверточной сети на lasagne для классификации датасета cifar10. Брал отсюда общую структуру сети на lasagne с theano.
https://github.com/ml-mipt/ml-mipt-part1/blob/master/2017/home_work/hw6/cifar.ipynb
2. Нейронная сеть на keras для классификации датасета cifar10. Подсмотрел тут конкруцию самой сети.
http://parneetk.github.io/blog/cnn-cifar10/
3. Нейронные сети на torch и lasagne для классификации картинок cifar10. Брал отсюда некоторые идеи, которые помогли улучшить качество.
http://torch.ch/blog/2015/07/30/cifar.html
https://github.com/Lasagne/Recipes/blob/master/papers/deep_residual_learning/Deep_Residual_Learning_CIFAR-10.py

In [2]:
import theano
import theano.tensor as T
import lasagne
from sklearn.model_selection import StratifiedShuffleSplit

from scipy.misc import imrotate
from scipy.stats import norm

def load_data():
    clocks = os.listdir('./clocks')
    crocodiles = os.listdir('./crocodiles')
    
    xs = []
    ys = []
    for image in clocks:
        with open('clocks/'+image, 'rb') as f:
            xs.append(plt.imread(f, 'F').transpose(2,0,1))    
            ys.append(0)
    for image in crocodiles:
        with open('crocodiles/'+image, 'rb') as f:
            xs.append(plt.imread(f, 'F').transpose(2,0,1))
            ys.append(1)
    
    for train_ind, test_ind in (StratifiedShuffleSplit(n_splits=1, random_state=0, test_size=0.2).split(xs, ys)):
        X_train = [xs[ind] for ind in train_ind]/ np.float32(255)
        Y_train = [ys[ind] for ind in train_ind]
        X_test = [xs[ind] for ind in test_ind]/ np.float32(255)
        Y_test = [ys[ind] for ind in test_ind]
    
    # добавим повернутые на рандомный угол изображения
    for image in clocks:
        with open('clocks/'+image, 'rb') as f:
            np.append(X_train, imrotate(plt.imread(f, 'F'), norm.rvs(0, 0.5), interp='nearest').transpose(2,0,1)/np.float32(255))
            np.append(Y_train, 0)
    for image in crocodiles:
        with open('crocodiles/'+image, 'rb') as f:
            np.append(X_train,imrotate(plt.imread(f, 'F'), norm.rvs(0, 0.5), interp='nearest').transpose(2,0,1)/np.float32(255))
            np.append(Y_train, 1)
    
    # создание зеркальных изображений
    X_train_flip = X_train[:,:,:,::-1]
    Y_train_flip = Y_train
    X_train = np.concatenate((X_train,X_train_flip),axis=0)
    Y_train = np.concatenate((Y_train,Y_train_flip),axis=0)
    
    return (
        lasagne.utils.floatX(X_train),
        Y_train,
        lasagne.utils.floatX(X_test),
        Y_test,
    )

Using cuDNN version 5110 on context None
Mapped name None to device cuda0: GRID K520 (0000:00:03.0)


In [3]:
X_train, y_train, X_test, y_test = load_data()
print("Классы разделились в тестовой выборке равномерно")
print(y_test.count(0))
print(y_test.count(1))

Классы разделились в тестовой выборке равномерно
100
100


### Создаем саму сеть

In [4]:
def build_cnn(input_var=None):
    net = lasagne.layers.InputLayer(shape=(None, 3, 32, 32), input_var=input_var)
    
    net = lasagne.layers.BatchNormLayer(net)
    net = lasagne.layers.Conv2DLayer(net, num_filters=32, filter_size=(3,3), stride=(1,1), pad='same', b=None)
    net = lasagne.layers.Conv2DLayer(net, num_filters=32, filter_size=(3,3), stride=(1,1), pad='same', b=None)
    net = lasagne.layers.MaxPool2DLayer(net, 2)
    net = lasagne.layers.DropoutLayer(net, p=0.4)
    
    net = lasagne.layers.BatchNormLayer(net)
    net = lasagne.layers.Conv2DLayer(net, num_filters=64, filter_size=(3,3), stride=(1,1), pad='same', b=None)
    net = lasagne.layers.Conv2DLayer(net, num_filters=64, filter_size=(3,3), stride=(1,1), pad='same', b=None)
    net = lasagne.layers.MaxPool2DLayer(net, 2)
    net = lasagne.layers.DropoutLayer(net, p=0.4)

    net = lasagne.layers.BatchNormLayer(net)
    net = lasagne.layers.Conv2DLayer(net, num_filters=128, filter_size=(3,3), stride=(1,1), pad='same', b=None)
    net = lasagne.layers.Conv2DLayer(net, num_filters=128, filter_size=(3,3), stride=(1,1), pad='same', b=None)
    net = lasagne.layers.MaxPool2DLayer(net, 2)
    net = lasagne.layers.DropoutLayer(net, p=0.4)
    
    net = lasagne.layers.BatchNormLayer(net)
    net = lasagne.layers.Conv2DLayer(net, num_filters=256, filter_size=(5,5), stride=(1,1), pad='same', b=None)
    net = lasagne.layers.Conv2DLayer(net, num_filters=256, filter_size=(5,5), stride=(1,1), pad='same', b=None)
    net = lasagne.layers.MaxPool2DLayer(net, 2)
    net = lasagne.layers.DropoutLayer(net, p=0.5)
    
    net = lasagne.layers.DenseLayer(net, num_units=512, nonlinearity=lasagne.nonlinearities.elu)
    net = lasagne.layers.DenseLayer(net, num_units=256, nonlinearity=lasagne.nonlinearities.elu)
    
    net = lasagne.layers.DenseLayer(net, num_units=2, nonlinearity=lasagne.nonlinearities.softmax)

    return net

In [5]:
input_X = T.tensor4("X")
target_y = T.vector("target Y integer",dtype='int32')

In [6]:
net = build_cnn(input_X)

In [7]:
y_predicted = lasagne.layers.get_output(net)
all_weights = lasagne.layers.get_all_params(net, trainable=True)
print(all_weights)

[beta, gamma, W, W, beta, gamma, W, W, beta, gamma, W, W, beta, gamma, W, W, W, b, W, b, W, b]


#### Используем l2 регуляризацию, чтобы сеть не переобучалась так сильно. В качестве метода оптимизации используем momentum. Он быстрее сходится, чем adam, а так же показывает немного лучше результат.

In [8]:
from lasagne.layers import get_all_layers
loss = lasagne.objectives.categorical_crossentropy(y_predicted, target_y).mean()
loss += lasagne.regularization.regularize_layer_params(get_all_layers(net), lasagne.regularization.l2) * 0.0001
accuracy = lasagne.objectives.categorical_accuracy(y_predicted, target_y).mean()

In [9]:
curr_lr = 0.01
updates = lasagne.updates.momentum(loss, all_weights, learning_rate=curr_lr, momentum=0.9)

In [10]:
train_fun = theano.function([input_X, target_y], [loss, accuracy], updates=updates, allow_input_downcast=True)
accuracy_fun = theano.function([input_X, target_y], accuracy, allow_input_downcast=True)

### Batch iterator

Также мы здесь шафлим батчи, это улучшает качество предсказания.

In [11]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

### Обучение сети.
Learning rate уменьшается на 12 эпохе, это число взято из экспериментов и наблюдения за процессом обучения:)


In [12]:
import time

num_epochs = 30 
batch_size = 64

for epoch in range(num_epochs):
    if epoch == 11:
        curr_lr = curr_lr/2
        updates = lasagne.updates.momentum(loss, all_weights, learning_rate=curr_lr, momentum=0.9)
        train_fun = theano.function([input_X, target_y], [loss, accuracy], updates=updates, allow_input_downcast=True)    

    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batch_size, True):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_test, y_test, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    print("Epoch {} of {} took {:.3f}s".format(epoch+1, num_epochs, time.time() - start_time))
    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(val_acc / val_batches * 100))

Epoch 1 of 30 took 10.398s
  training loss (in-iteration):		0.953824
  train accuracy:		69.88 %
  validation accuracy:		81.25 %
Epoch 2 of 30 took 10.367s
  training loss (in-iteration):		0.723020
  train accuracy:		83.81 %
  validation accuracy:		82.29 %
Epoch 3 of 30 took 10.387s
  training loss (in-iteration):		0.673545
  train accuracy:		86.81 %
  validation accuracy:		86.98 %
Epoch 4 of 30 took 10.382s
  training loss (in-iteration):		0.643681
  train accuracy:		88.56 %
  validation accuracy:		87.50 %
Epoch 5 of 30 took 10.382s
  training loss (in-iteration):		0.592385
  train accuracy:		89.75 %
  validation accuracy:		84.38 %
Epoch 6 of 30 took 16.905s
  training loss (in-iteration):		0.587309
  train accuracy:		89.50 %
  validation accuracy:		86.98 %
Epoch 7 of 30 took 20.636s
  training loss (in-iteration):		0.588497
  train accuracy:		90.25 %
  validation accuracy:		85.42 %
Epoch 8 of 30 took 20.672s
  training loss (in-iteration):		0.546800
  train accuracy:		92.44 %
  valida

KeyboardInterrupt: 

### Посмотрим на итоговый результат сети.

In [36]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 50):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))


Final results:
  test accuracy:		93.00 %


Получили 93%, то есть сеть правильно классифицировала 188 картинок из тестовой выборки:)